# Gather Hourly Traffic Data <br>
*From statens vegvesen API: https://www.vegvesen.no/trafikkdata/api*

Import necessary packages:

In [1]:
import os
import json
import requests
import time
import random
from tqdm import tqdm  
import pandas as pd
from datetime import datetime
# Display all columns in the df
pd.set_option('display.max_columns', None)

Set paths etc.:

In [2]:
# define the base path where the file should be saved
base_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData'
# define the file name
file_name = "all_IDs.json"
# combine the base path and file name to create a valid file path
file_path = os.path.join(base_path, file_name)

### First query (get traffic point ID's):

In [3]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

# define the GraphQL query for fetching ID's of registration points (only Europe roads)
query = """
{
  trafficRegistrationPoints(searchQuery: {roadCategoryIds: [E ,R, F], registrationFrequency: CONTINUOUS}) {
    id
  }
}
"""

# make a POST request to the API
response = requests.post(endpoint, json={"query": query})

# check the status code of the response
if response.status_code == 200:
    # if the request was successful, get the data from the response
    data = response.json()

    # open the file for writing
    with open(file_path, "w") as file:
        # write the data to the file in JSON format
        json.dump(data, file)

Store query number 1 and convert ID's to list for inserting in query number 2:

In [4]:
# define the file path of the JSON file
file_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/all_IDs.json'

# open the file for reading
with open(file_path, "r") as file:
    # load the data from the file into a Python object
    data = json.load(file)

# create an empty list for the registration point ID's
registration_point_ids = []

# iterate over the data returned by the GraphQL query
for item in data["data"]["trafficRegistrationPoints"]:
    # get the ID of the current registration point
    id = item["id"]

    # add the ID to the list of registration point ID's
    registration_point_ids.append(id)

### Second query (get traffic volume for every hour in a one-week period): <br>
*NB! This query must be run **twice** because it only possible to get 100 hours per query.* <br>
*The dataframes must therefore be merged together later on.*

In [5]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_1 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-09-17T00:00:00+02:00"
        to: "2018-09-20T11:00:00+02:00"  
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_1 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_1["registration_point_id"] = registration_point_id

      data_1.append(volume_data_1)

      pbar.update(1)

100%|██████████| 3126/3126 [1:48:07<00:00,  2.08s/it]


*Second run:*

In [6]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_2 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-09-20T11:00:00+02:00"
        to: "2018-09-24T00:00:00+02:00"
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_2 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_2["registration_point_id"] = registration_point_id

      data_2.append(volume_data_2)

      pbar.update(1)

100%|██████████| 3126/3126 [1:48:52<00:00,  2.09s/it]


Store data collected from query number 2:

In [7]:
# define the path and filename for the JSON file
filename_1 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Sep.json'
# write the data to the JSON file
with open(filename_1, "w") as f:
    json.dump(data_1, f, indent=2) # usually indent = 4, but ok 

# define the path and filename for the JSON file
filename_2 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Sep.json'
# write the data to the JSON file
with open(filename_2, "w") as f:
    json.dump(data_2, f, indent=2) 

Open data collected from query number 2:

In [8]:
with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Sep.json', 'r') as file:
    json_data_1 = json.load(file)

with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Sep.json', 'r') as file:
    json_data_2 = json.load(file)

### def flatten_json <br>
*Takes the data gathered from query 2 (stored as a json-file), flattening the data and converts it to a pandas dataframe.*

In [9]:
def flatten_json(json_data):
    flat_list = []
    for item in json_data:
        edges = item.get("data", {}).get("trafficData", {}).get("volume", {}).get("byHour", {}).get("edges", [])
        traffic_registration_point_id = item.get("registration_point_id")

        data_count = 0
        temp_list = []
        for edge in edges:
            node = edge.get("node", {})
            total = node.get("total", {})

            if total is None:
                continue

            volume_numbers = total.get("volumeNumbers") or {}
            coverage = total.get("coverage") or {}

            if volume_numbers.get("volume") is None:
                continue

            from_datetime = datetime.fromisoformat(node.get("from").replace("+02:00", ""))
            to_datetime = datetime.fromisoformat(node.get("to").replace("+02:00", ""))

            flat_dict = {
                "traffic_point_id": traffic_registration_point_id,
                "year": from_datetime.year,
                "month": from_datetime.month,
                "day": from_datetime.day,
                "from_hour": from_datetime.strftime("%H:%M"),
                "to_hour": to_datetime.strftime("%H:%M"),
                "volume": volume_numbers.get("volume"),
                "coverage_percentage": coverage.get("percentage")
            }
            data_count += 1
            temp_list.append(flat_dict)

        if data_count >= 83:    # 83 er riktig tall egentlig            # NB! Denne burde egentlig være "if data_count == 168:", men det ser ut til at det kun er mulig å hente ut 100 timer på ett punkt om gangen.
                                            # data_count == 168 fordi 24 timer i 7 dager (24*7) = 168
            flat_list.extend(temp_list)

    return flat_list


**Read warning in function above.**

Run function, check output:

In [10]:
flat_data_1 = flatten_json(json_data_1)
flat_data_2 = flatten_json(json_data_2)
# Convert flattened data to DataFrame
volume_df_1 = pd.DataFrame(flat_data_1)
volume_df_2 = pd.DataFrame(flat_data_2)
display(volume_df_1)#[:50]
display(volume_df_2)#[:50]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,17,00:00,01:00,20,100.0
1,65823V1668921,2018,9,17,01:00,02:00,6,100.0
2,65823V1668921,2018,9,17,02:00,03:00,1,100.0
3,65823V1668921,2018,9,17,03:00,04:00,0,100.0
4,65823V1668921,2018,9,17,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
160185,10795V320297,2018,9,20,06:00,07:00,395,100.0
160186,10795V320297,2018,9,20,07:00,08:00,755,100.0
160187,10795V320297,2018,9,20,08:00,09:00,657,100.0
160188,10795V320297,2018,9,20,09:00,10:00,453,100.0


,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,20,11:00,12:00,62,100.0
1,65823V1668921,2018,9,20,12:00,13:00,68,100.0
2,65823V1668921,2018,9,20,13:00,14:00,53,100.0
3,65823V1668921,2018,9,20,14:00,15:00,69,100.0
4,65823V1668921,2018,9,20,15:00,16:00,56,100.0
...,...,...,...,...,...,...,...,...
167329,10795V320297,2018,9,23,19:00,20:00,525,100.0
167330,10795V320297,2018,9,23,20:00,21:00,357,100.0
167331,10795V320297,2018,9,23,21:00,22:00,256,100.0
167332,10795V320297,2018,9,23,22:00,23:00,165,100.0


### Merge the two dataframes:

In [11]:
# Concatenate the two dataframes
merged_df = pd.concat([volume_df_1, volume_df_2], ignore_index=True)

# Group by traffic_point_id and filter based on the number of data points (168 hours)
completeVolume_df = merged_df.groupby('traffic_point_id').filter(lambda x: len(x) >= 166)

# Reset the index of the filtered dataframe
completeVolume_df.reset_index(drop=True, inplace=True)


Check the merged dataframe - looks good:

In [12]:
completeVolume_df#[:60]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,9,17,00:00,01:00,20,100.0
1,65823V1668921,2018,9,17,01:00,02:00,6,100.0
2,65823V1668921,2018,9,17,02:00,03:00,1,100.0
3,65823V1668921,2018,9,17,03:00,04:00,0,100.0
4,65823V1668921,2018,9,17,04:00,05:00,4,100.0
...,...,...,...,...,...,...,...,...
320692,10795V320297,2018,9,23,19:00,20:00,525,100.0
320693,10795V320297,2018,9,23,20:00,21:00,357,100.0
320694,10795V320297,2018,9,23,21:00,22:00,256,100.0
320695,10795V320297,2018,9,23,22:00,23:00,165,100.0


Save dataframe as CSV-file in directory:

In [13]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Sep.csv"

# Convert the dataframe to a CSV file and save it to the specified path
completeVolume_df.to_csv(output_file_path, index=False)


In [14]:
print(completeVolume_df['traffic_point_id'].nunique() / len(registration_point_ids))

0.610684580934101


In [15]:
#completeVolume_df = pd.read_csv('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_July.csv')
#completeVolume_df

### Pivot the dataframe <br> 
*This is to display hourly volume on the y-axis (not sure if necessary)*

In [16]:
# Create a new column 'timestamp' combining year, month, day, and from_hour
completeVolume_df['timestamp'] = completeVolume_df['year'].astype(str) + '-' + completeVolume_df['month'].astype(str).str.zfill(2) + '-' + completeVolume_df['day'].astype(str).str.zfill(2) + ' ' + completeVolume_df['from_hour']

# Pivot the DataFrame using 'traffic_point_id' as index and 'timestamp' as columns
pivoted_df = completeVolume_df.pivot_table(index='traffic_point_id', columns='timestamp', values='volume')

# Reset index and fill missing values with 0 (if any)
pivoted_df.reset_index(inplace=True)
pivoted_df.fillna(0, inplace=True)

# Display pivoted dataframe
pivoted_df

timestamp,traffic_point_id,2018-09-17 00:00,2018-09-17 01:00,2018-09-17 02:00,2018-09-17 03:00,2018-09-17 04:00,2018-09-17 05:00,2018-09-17 06:00,2018-09-17 07:00,2018-09-17 08:00,2018-09-17 09:00,2018-09-17 10:00,2018-09-17 11:00,2018-09-17 12:00,2018-09-17 13:00,2018-09-17 14:00,2018-09-17 15:00,2018-09-17 16:00,2018-09-17 17:00,2018-09-17 18:00,2018-09-17 19:00,2018-09-17 20:00,2018-09-17 21:00,2018-09-17 22:00,2018-09-17 23:00,2018-09-18 00:00,2018-09-18 01:00,2018-09-18 02:00,2018-09-18 03:00,2018-09-18 04:00,2018-09-18 05:00,2018-09-18 06:00,2018-09-18 07:00,2018-09-18 08:00,2018-09-18 09:00,2018-09-18 10:00,2018-09-18 11:00,2018-09-18 12:00,2018-09-18 13:00,2018-09-18 14:00,2018-09-18 15:00,2018-09-18 16:00,2018-09-18 17:00,2018-09-18 18:00,2018-09-18 19:00,2018-09-18 20:00,2018-09-18 21:00,2018-09-18 22:00,2018-09-18 23:00,2018-09-19 00:00,2018-09-19 01:00,2018-09-19 02:00,2018-09-19 03:00,2018-09-19 04:00,2018-09-19 05:00,2018-09-19 06:00,2018-09-19 07:00,2018-09-19 08:00,2018-09-19 09:00,2018-09-19 10:00,2018-09-19 11:00,2018-09-19 12:00,2018-09-19 13:00,2018-09-19 14:00,2018-09-19 15:00,2018-09-19 16:00,2018-09-19 17:00,2018-09-19 18:00,2018-09-19 19:00,2018-09-19 20:00,2018-09-19 21:00,2018-09-19 22:00,2018-09-19 23:00,2018-09-20 00:00,2018-09-20 01:00,2018-09-20 02:00,2018-09-20 03:00,2018-09-20 04:00,2018-09-20 05:00,2018-09-20 06:00,2018-09-20 07:00,2018-09-20 08:00,2018-09-20 09:00,2018-09-20 10:00,2018-09-20 11:00,2018-09-20 12:00,2018-09-20 13:00,2018-09-20 14:00,2018-09-20 15:00,2018-09-20 16:00,2018-09-20 17:00,2018-09-20 18:00,2018-09-20 19:00,2018-09-20 20:00,2018-09-20 21:00,2018-09-20 22:00,2018-09-20 23:00,2018-09-21 00:00,2018-09-21 01:00,2018-09-21 02:00,2018-09-21 03:00,2018-09-21 04:00,2018-09-21 05:00,2018-09-21 06:00,2018-09-21 07:00,2018-09-21 08:00,2018-09-21 09:00,2018-09-21 10:00,2018-09-21 11:00,2018-09-21 12:00,2018-09-21 13:00,2018-09-21 14:00,2018-09-21 15:00,2018-09-21 16:00,2018-09-21 17:00,2018-09-21 18:00,2018-09-21 19:00,2018-09-21 20:00,2018-09-21 21:00,2018-09-21 22:00,2018-09-21 23:00,2018-09-22 00:00,2018-09-22 01:00,2018-09-22 02:00,2018-09-22 03:00,2018-09-22 04:00,2018-09-22 05:00,2018-09-22 06:00,2018-09-22 07:00,2018-09-22 08:00,2018-09-22 09:00,2018-09-22 10:00,2018-09-22 11:00,2018-09-22 12:00,2018-09-22 13:00,2018-09-22 14:00,2018-09-22 15:00,2018-09-22 16:00,2018-09-22 17:00,2018-09-22 18:00,2018-09-22 19:00,2018-09-22 20:00,2018-09-22 21:00,2018-09-22 22:00,2018-09-22 23:00,2018-09-23 00:00,2018-09-23 01:00,2018-09-23 02:00,2018-09-23 03:00,2018-09-23 04:00,2018-09-23 05:00,2018-09-23 06:00,2018-09-23 07:00,2018-09-23 08:00,2018-09-23 09:00,2018-09-23 10:00,2018-09-23 11:00,2018-09-23 12:00,2018-09-23 13:00,2018-09-23 14:00,2018-09-23 15:00,2018-09-23 16:00,2018-09-23 17:00,2018-09-23 18:00,2018-09-23 19:00,2018-09-23 20:00,2018-09-23 21:00,2018-09-23 22:00,2018-09-23 23:00
0,00000V1702725,5.0,4.0,0.0,2.0,2.0,21.0,132.0,433.0,436.0,469.0,624.0,801.0,862.0,868.0,952.0,1243.0,1268.0,1003.0,936.0,857.0,525.0,252.0,45.0,16.0,12.0,4.0,2.0,5.0,5.0,18.0,108.0,399.0,399.0,402.0,582.0,683.0,764.0,756.0,902.0,1062.0,1197.0,928.0,809.0,699.0,446.0,237.0,41.0,20.0,8.0,0.0,0.0,2.0,5.0,16.0,122.0,426.0,419.0,474.0,647.0,794.0,879.0,952.0,942.0,1223.0,1258.0,978.0,903.0,810.0,531.0,264.0,59.0,19.0,10.0,5.0,2.0,5.0,0.0,20.0,128.0,449.0,435.0,462.0,650.0,814.0,952.0,1030.0,965.0,1268.0,1274.0,995.0,992.0,874.0,626.0,268.0,56.0,10.0,12.0,4.0,10.0,1.0,4.0,22.0,121.0,375.0,415.0,469.0,849.0,984.0,1031.0,1082.0,1044.0,1069.0,790.0,566.0,469.0,396.0,310.0,211.0,37.0,21.0,9.0,3.0,3.0,2.0,3.0,4.0,20.0,37.0,88.0,335.0,809.0,1373.0,1625.0,1665.0,1846.0,1756.0,1513.0,1229.0,846.0,437.0,101.0,41.0,23.0,15.0,17.0,9.0,5.0,1.0,2.0,1.0,4.0,3.0,11.0,33.0,37.0,85.0,133.0,157.0,235.0,212.0,228.0,198.0,136.0,106.0,96.0,52.0,32.0,17.0
1,00000V1702751,243.0,82.0,51.0,56.0,131.0,624.0,1721.0,3305.0,3064.0,2398.0,2562.0,2859.0,2757.0,2929.0,3260.0,4415.0,3929.0,2779.0,2496.0,2062.0,1497.0,1024.0,526.0,390.0,1

Store new pivoted dataframe:

In [17]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Sep_pivoted.csv"

# Convert the dataframe to a CSV file and save it to the specified path
pivoted_df.to_csv(output_file_path, index=False)